In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# IMDB movie review


In [0]:
import pandas as pd

data = pd.read_csv("/content/drive/My Drive/lecture/l_d/p9/IMDB-Movie-Data.csv")

In [5]:
data.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


In [25]:
data.shape

(1000, 13)

In [8]:
rating_mean = data['Rating'].mean()
rating_mean

6.723200000000003

In [10]:
vote_quantile = data["Votes"].quantile(0.9)
vote_quantile

406251.10000000003

In [15]:
quantile_movies = data.copy().loc[data["Votes"] >= vote_quantile]
quantile_movies.shape

(100, 12)

In [0]:
def weighted_rate(data, vote_quantile=vote_quantile, rating_mean=rating_mean):
    votes = data["Votes"]
    ratings = data["Rating"]
    return ((votes/(votes + vote_quantile)*ratings) +
            (vote_quantile/(votes + vote_quantile)*rating_mean))

In [0]:
data['score'] = data.apply(weighted_rate, axis=1)

In [0]:
col = ["Title", "Rating", "Votes", "score"]
rs_original = data.sort_values("Rating", ascending=False)[col]
rs_new = data.sort_values("score", ascending=False)[col]

In [38]:
rs_original.shape

(1000, 4)

In [39]:
rs_original.head()

,Title,Rating,Votes,score
54,The Dark Knight,9.0,1791916,8.579216
80,Inception,8.8,1583625,8.376003
117,Dangal,8.8,48969,6.946606
36,Interstellar,8.6,1047747,8.075617
96,Kimi no na wa,8.6,34110,6.868575


In [34]:
rs_new.shape

(1000, 4)

In [40]:
rs_new.head()

,Title,Rating,Votes,score
54,The Dark Knight,9.0,1791916,8.579216
80,Inception,8.8,1583625,8.376003
36,Interstellar,8.6,1047747,8.075617
124,The Dark Knight Rises,8.5,1222645,8.056861
99,The Departed,8.5,937414,7.962793


In [41]:
data["Description"][0]

'A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe.'

In [42]:
data["Description"].head()

0    A group of intergalactic criminals are forced ...
1    Following clues to the origin of mankind, a te...
2    Three girls are kidnapped by a man with a diag...
3    In a city of humanoid animals, a hustling thea...
4    A secret government agency recruits some of th...
Name: Description, dtype: object

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(stop_words='english')

data.Description = data.Description.fillna('')

tfidf_matrix = tfidf.fit_transform(data.Description)
tfidf_matrix.shape

(1000, 5667)

In [48]:
from sklearn.metrics.pairwise import linear_kernel


cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(1000, 1000)

In [53]:
indices = pd.Series(data.index, index=data["Title"]).drop_duplicates()
indices.head()

Title
Guardians of the Galaxy    0
Prometheus                 1
Split                      2
Sing                       3
Suicide Squad              4
dtype: int64

In [0]:
def rs(title, n=10, sim=cosine_sim):
    idx = indices[title]
    sim_score = list(enumerate(cosine_sim[idx]))
    sim_score = sorted(sim_score, key=lambda x : x[1], reverse=True)
    sim_score = sim_score[1:(n + 1)]
    movie_index = [i[0] for i in sim_score]
    return data.Title.iloc[movie_index]

In [65]:
rs("The Dark Knight Rises")

54              The Dark Knight
144            Django Unchained
522                      Viking
471            Marie Antoinette
254               The Conjuring
920                   Centurion
253    The Amazing Spider-Man 2
604       The Rise of the Krays
615                Bastille Day
915                     Goksung
Name: Title, dtype: object